In [1]:
import numpy as np
import pandas as pd

In [2]:
c

In [3]:
df.head()

,Track,Artist,Album,Year,Duration,Time_Signature,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Popularity,Decade
0,Play A Simple Song,38 Special,38 Special,1977,193,4,0.521,0.367,0,-13.866,1,0.0278,0.6920,0.000003,0.1080,0.789,83.412,16,1970s
1,Four Wheels,38 Special,38 Special,1977,283,4,0.535,0.710,2,-12.287,1,0.0428,0.0100,0.023000,0.0495,0.445,160.361,10,1970s
2,Fly Away,38 Special,38 Special,1977,313,4,0.563,0.563,2,-10.781,1,0.0263,0.0357,0.001850,0.1400,0.564,106.739,13,1970s
3,Tell Everybody,38 Special,38 Special,1977,249,4,0.638,0.694,11,-10.206,0,0.0310,0.1610,0.000034,0.0908,0.936,124.962,10,1970s
4,Just Wanna Rock & Roll,38 Special,38 Special,1977,357,4,0.388,0.701,2,-9.984,1,0.0360,0.0130,0.042200,0.1150,0.769,126.769,11,1970s


## Preprocessing

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [5]:
X = df.drop(columns=["Energy"])
y = df["Energy"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [6]:

# make_column_selector

num_attributes = [
    "Duration", 
    "Danceability", 
    "Key", 
    "Loudness", 
    "Speechiness", 
    "Acousticness", 
    "Instrumentalness", 
    "Liveness", 
    "Valence",
    "Tempo",
    "Popularity"
]

cat_attributes = ["Key", "Mode", "Decade"]

In [7]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("standarizer", StandardScaler()),
])

In [8]:
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore")),
])

In [9]:
preprocessing = ColumnTransformer([
    ("numerical", num_pipeline, num_attributes),
    ("categorical", cat_pipeline, cat_attributes),
])

In [10]:
X_train_prep = preprocessing.fit_transform(X_train)
X_test_prep = preprocessing.fit_transform(X_test)

## Training Models

### Linear Regression

In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
linear_regression = LinearRegression()

linear_regression.fit(X_train_prep, y_train)

LinearRegression()

### Decision Tree

In [13]:
from sklearn.tree import DecisionTreeRegressor

In [14]:
tree_regressor = DecisionTreeRegressor(max_depth=2)
tree_regressor.fit(X_train_prep, y_train)

DecisionTreeRegressor(max_depth=2)

### SVM

In [15]:
from sklearn.svm import LinearSVR

In [16]:
svm_regressor = LinearSVR(epsilon=0.5)
svm_regressor.fit(X_train_prep, y_train)

/home/jeremi/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVR(epsilon=0.5)

## Evaluating Models

## Linear Regression with Numpy

In [17]:
theta_best = np.linalg.inv(X_train_prep.T @ X_train_prep) @ X_train_prep.T @ y_train

In [18]:
theta_best

array([ 9.06943050e-02, -5.14029127e-01,  3.54330104e+13,  5.01634601e-01,
        3.55522042e-01, -3.15873897e-01,  2.38291251e-02, -1.62134542e-01,
       -7.18210539e-02, -2.29048915e-01, -2.37847393e-01,  5.22101543e+13,
        4.21035498e+13,  3.19969454e+13,  2.18903409e+13,  1.17837365e+13,
        1.67713204e+12, -8.42947241e+12, -1.85360769e+13, -2.86426813e+13,
       -3.87492858e+13, -4.88558902e+13, -5.89624947e+13, -2.98168816e-02,
       -1.96029002e-01,  8.84173029e-01,  9.93625127e-02,  9.69744653e-02,
        8.86844827e-01,  3.08661473e-01,  5.39980420e-01,  5.04443602e-01])

In [19]:
y_predict = X_test_prep @ theta_best

In [20]:
y_predict

array([ 3.66333974e+10, -1.53541692e+11,  7.46684152e+10, ...,
        3.66333974e+10,  1.12703433e+11,  1.88773469e+11])

### Using gradient descent

In [14]:
eta = 0.01  # Szybkość uczenia
n_epochs = 1000  # Liczba epok
batch_size = 32  # Rozmiar batcha
m = len(X_train)  # Liczba próbek

y_train_2 = y_train.to_numpy()

theta = np.random.randn(X_train_prep.shape[1], 1)


def mse(X, y, theta):
    m = len(y)
    predictions = X @ theta
    cost = (1 / (2 * m)) * np.sum((predictions - y) ** 2)
    return cost

for epoch in range(n_epochs):
    for i in range(0, m, batch_size):
        X_batch = X_train_prep[i:i + batch_size]
        y_batch = y_train_2[i:i + batch_size]

        gradients = (1 / batch_size) * X_batch.T @ (X_batch @ theta - y_batch)
        
        theta -= eta * gradients

    cost = mse(X_train, y_train, theta)
    print(f"Epoch {epoch+1}/{n_epochs}, Cost: {cost}")

ValueError: non-broadcastable output operand with shape (32,1) doesn't match the broadcast shape (32,32)

In [22]:
theta

array([[-1.53771799],
       [ 0.27136744],
       [ 1.99445907],
       [ 0.57771074],
       [ 0.54220233],
       [ 0.06467229],
       [ 1.47326036],
       [ 0.36860996],
       [ 0.42722269],
       [ 0.11635823],
       [ 0.25177219],
       [-1.65598632],
       [-0.59860078],
       [-0.06348978],
       [-0.22546767],
       [-0.01269909],
       [-0.11335285],
       [-0.99104498]])